## Exploring Equity Classifications with Machine Learning
### Proposal
DATE: May 20, 2021

TO: Annette Demchur, Rebecca Morgan

FROM: Margaret Atkinson

RE: Staff Initiated Study Proposal:

#### Exploring Equity Classifications with Machine Learning

I would like to conduct research to answer the following question: Could an
unsupervised machine learning algorithm create groups of towns based on
demographic information that would be useful to explore questions about equity? To
explain, when we compare all towns that pass the minority threshold (or TAZs or block
groups etc.) to all towns that do not, we may be missing the way the demographic
variables interact and a multifactored grouping could allow us to explore the
demographic towns with a more detailed approach without the blending nature of an
index.

This project would use python and specifically the Scikit-Learn python library to
conduct unsupervised machine learning based on demographic data at the town level.
The data would be demographic Census data from the American Community Survey 5-
year estimates at minimum on the topics of: Race/Ethnicity, Limited English Proficiency,
Median Income, Low Income, No Car Households, Population Density, Children, and
Seniors. The product would be a geographic file that shows groupings of towns by
demographic profile as found by the unsupervised machine learning algorithm as well
as a written description of what each grouping represents.

If the question is pursued and the results are useful - the ultimate intention (as a
follow up project) would be to look at the way MPO distributes funding between the
groups and within each group to look for disparities. Explanations of disparities could
lead to a re-examination of the variables used in the algorithm in order to provide
an additional check for equitable spending.


Towns in the MPO
BRMPO <- c("Beverly","Boston","Braintree","Cambridge","Chelsea","Everett","Framingham","Franklin","Gloucester","Lynn","Malden","Marlborough","Medford","Melrose","Newton","Peabody","Quincy","Revere","Salem","Somerville","Waltham","Watertown","Weymouth","Woburn","Acton","Arlington","Ashland","Bedford","Bellingham","Belmont","Bolton","Boxborough","Brookline","Burlington","Canton","Carlisle","Cohasset","Concord","Danvers","Dedham","Dover","Essex","Foxborough","Hamilton","Hingham","Holbrook","Holliston","Hopkinton","Hudson","Hull","Ipswich”,"Lexington","Lincoln","Littleton","Lynnfield","Manchester-by-the-Sea","Marblehead","Marshfield","Maynard","Medfield",           "Medway","Middleton","Milford","Millis","Milton","Nahant","Natick","Needham","Norfolk","North Reading","Norwell","Norwood","Randolph","Reading","Rockland","Rockport","Saugus","Scituate","Sharon","Sherborn","Southborough","Stoneham","Stow","Sudbury","Swampscott","Topsfield","Wakefield","Walpole","Wayland","Wellesley","Wenham","Weston","Westwood","Wilmington","Winchester","Winthrop","Wrentham")

Algorithms
K-means with K-means++ as initialization of seeding
Only deterministic if the seeds are always the same, which K-means++ does not guarantee.
DBSCAN


Spectral Clustering
Fully deterministic
